# Artificial Intelligence Engineer Nanodegree - Probabilistic Models
## Project: Sign Language Recognition System
- [Introduction](#intro)
- [Part 1 Feature Selection](#part1_tutorial)
    - [Tutorial](#part1_tutorial)
    - [Features Submission](#part1_submission)
    - [Features Unittest](#part1_test)
- [Part 2 Train the models](#part2_tutorial)
    - [Tutorial](#part2_tutorial)
    - [Model Selection Score Submission](#part2_submission)
    - [Model Score Unittest](#part2_test)
- [Part 3 Build a Recognizer](#part3_tutorial)
    - [Tutorial](#part3_tutorial)
    - [Recognizer Submission](#part3_submission)
    - [Recognizer Unittest](#part3_test)
- [Part 4 (OPTIONAL) Improve the WER with Language Models](#part4_info)

<a id='intro'></a>
## Introduction
The overall goal of this project is to build a word recognizer for American Sign Language video sequences, demonstrating the power of probabalistic models.  In particular, this project employs  [hidden Markov models (HMM's)](https://en.wikipedia.org/wiki/Hidden_Markov_model) to analyze a series of measurements taken from videos of American Sign Language (ASL) collected for research (see the [RWTH-BOSTON-104 Database](http://www-i6.informatik.rwth-aachen.de/~dreuw/database-rwth-boston-104.php)).  In this video, the right-hand x and y locations are plotted as the speaker signs the sentence.
[![ASLR demo](http://www-i6.informatik.rwth-aachen.de/~dreuw/images/demosample.png)](https://drive.google.com/open?id=0B_5qGuFe-wbhUXRuVnNZVnMtam8)

The raw data, train, and test sets are pre-defined.  You will derive a variety of feature sets (explored in Part 1), as well as implement three different model selection criterion to determine the optimal number of hidden states for each word model (explored in Part 2). Finally, in Part 3 you will implement the recognizer and compare the effects the different combinations of feature sets and model selection criteria.  

At the end of each Part, complete the submission cells with implementations, answer all questions, and pass the unit tests.  Then submit the completed notebook for review!

<a id='part1_tutorial'></a>
## PART 1: Data

### Features Tutorial
##### Load the initial database
A data handler designed for this database is provided in the student codebase as the `AslDb` class in the `asl_data` module.  This handler creates the initial [pandas](http://pandas.pydata.org/pandas-docs/stable/) dataframe from the corpus of data included in the `data` directory as well as dictionaries suitable for extracting data in a format friendly to the [hmmlearn](https://hmmlearn.readthedocs.io/en/latest/) library.  We'll use those to create models in Part 2.

To start, let's set up the initial database and select an example set of features for the training set.  At the end of Part 1, you will create additional feature sets for experimentation. 

In [1]:
import numpy as np
import pandas as pd
from asl_data import AslDb

In [ ]:
asl = AslDb() # initializes the database
asl.df.head() # displays the first five rows of the asl database, indexed by video and frame

In [ ]:
asl.df.ix[98,1]  # look at the data available for an individual frame

The frame represented by video 98, frame 1 is shown here:
![Video 98](http://www-i6.informatik.rwth-aachen.de/~dreuw/database/rwth-boston-104/overview/images/orig/098-start.jpg)

##### Feature selection for training the model
The objective of feature selection when training a model is to choose the most relevant variables while keeping the model as simple as possible, thus reducing training time.  We can use the raw features already provided or derive our own and add columns to the pandas dataframe `asl.df` for selection. As an example, in the next cell a feature named `'grnd-ry'` is added. This feature is the difference between the right-hand y value and the nose y value, which serves as the "ground" right y value. 

In [ ]:
asl.df['grnd-ry'] = asl.df['right-y'] - asl.df['nose-y']
asl.df.head()  # the new feature 'grnd-ry' is now in the frames dictionary

##### Try it!

In [ ]:
from asl_utils import test_features_tryit

asl.df['grnd-rx'] = asl.df['right-x'] - asl.df['nose-x']
asl.df['grnd-ly'] = asl.df['left-y'] - asl.df['nose-y']
asl.df['grnd-lx'] = asl.df['left-x'] - asl.df['nose-x']
test_features_tryit(asl)

In [ ]:
# collect the features into a list
features_ground = ['grnd-rx','grnd-ry','grnd-lx','grnd-ly']
 #show a single set of features for a given (video, frame) tuple
[asl.df.ix[98,1][v] for v in features_ground]

##### Build the training set
Now that we have a feature list defined, we can pass that list to the `build_training` method to collect the features for all the words in the training set.  Each word in the training set has multiple examples from various videos.  Below we can see the unique words that have been loaded into the training set:

In [ ]:
training = asl.build_training(features_ground)
print("Training words: {}".format(training.words))

The training data in `training` is an object of class `WordsData` defined in the `asl_data` module.  in addition to the `words` list, data can be accessed with the `get_all_sequences`, `get_all_Xlengths`, `get_word_sequences`, and `get_word_Xlengths` methods. We need the `get_word_Xlengths` method to train multiple sequences with the `hmmlearn` library.  In the following example, notice that there are two lists; the first is a concatenation of all the sequences(the X portion) and the second is a list of the sequence lengths(the Lengths portion).

In [ ]:
training.get_word_Xlengths('CHOCOLATE')

###### More feature sets
So far we have a simple feature set that is enough to get started modeling.  However, we might get better results if we manipulate the raw values a bit more, so we will go ahead and set up some other options now for experimentation later.  For example, we could normalize each speaker's range of motion with grouped statistics using [Pandas stats](http://pandas.pydata.org/pandas-docs/stable/api.html#api-dataframe-stats) functions and [pandas groupby](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.groupby.html).  Below is an example for finding the means of all speaker subgroups.

In [ ]:
df_means = asl.df.groupby('speaker').mean()
df_means

To select a mean that matches by speaker, use the pandas [map](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.map.html) method:

In [ ]:
asl.df['left-x-mean']= asl.df['speaker'].map(df_means['left-x'])
asl.df.head()

##### Try it!

In [ ]:
from asl_utils import test_std_tryit
# TODO Create a dataframe named `df_std` with standard deviations grouped by speaker
df_std = asl.df.groupby('speaker').std()

# test the code
test_std_tryit(df_std)

<a id='part1_submission'></a>
### Features Implementation Submission
Implement four feature sets and answer the question that follows.
- normalized Cartesian coordinates
    - use *mean* and *standard deviation* statistics and the [standard score](https://en.wikipedia.org/wiki/Standard_score) equation to account for speakers with different heights and arm length
    
- polar coordinates
    - calculate polar coordinates with [Cartesian to polar equations](https://en.wikipedia.org/wiki/Polar_coordinate_system#Converting_between_polar_and_Cartesian_coordinates)
    - use the [np.arctan2](https://docs.scipy.org/doc/numpy-1.10.0/reference/generated/numpy.arctan2.html) function and *swap the x and y axes* to move the $0$ to $2\pi$ discontinuity to 12 o'clock instead of 3 o'clock;  in other words, the normal break in radians value from $0$ to $2\pi$ occurs directly to the left of the speaker's nose, which may be in the signing area and interfere with results.  By swapping the x and y axes, that discontinuity move to directly above the speaker's head, an area not generally used in signing.

- delta difference
    - as described in Thad's lecture, use the difference in values between one frame and the next frames as features
    - pandas [diff method](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.diff.html) and [fillna method](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.fillna.html) will be helpful for this one

- custom features
    - These are your own design; combine techniques used above or come up with something else entirely. We look forward to seeing what you come up with! 
    Some ideas to get you started:
        - normalize using a [feature scaling equation](https://en.wikipedia.org/wiki/Feature_scaling)
        - normalize the polar coordinates
        - adding additional deltas


In [ ]:
# TODO add features for normalized by speaker values of left, right, x, y
# Name these 'norm-rx', 'norm-ry', 'norm-lx', and 'norm-ly'
# using Z-score scaling (X-Xmean)/Xstd

asl.df['left-x-mean']= asl.df['speaker'].map(df_means['left-x'])
asl.df['right-x-mean']= asl.df['speaker'].map(df_means['right-x'])
asl.df['left-y-mean']= asl.df['speaker'].map(df_means['left-y'])
asl.df['right-y-mean']= asl.df['speaker'].map(df_means['right-y'])

asl.df['left-x-std']= asl.df['speaker'].map(df_std['left-x'])
asl.df['right-x-std']= asl.df['speaker'].map(df_std['right-x'])
asl.df['left-y-std']= asl.df['speaker'].map(df_std['left-y'])
asl.df['right-y-std']= asl.df['speaker'].map(df_std['right-y'])

asl.df['norm-rx'] = (asl.df['right-x'] - asl.df['right-x-mean']) / asl.df['right-x-std']
asl.df['norm-ry'] = (asl.df['right-y'] - asl.df['right-y-mean']) / asl.df['right-y-std']
asl.df['norm-lx'] = (asl.df['left-x'] - asl.df['left-x-mean']) / asl.df['left-x-std']
asl.df['norm-ly'] = (asl.df['left-y'] - asl.df['left-y-mean']) / asl.df['left-y-std']

In [ ]:
# TODO add features for polar coordinate values where the nose is the origin
# Name these 'polar-rr', 'polar-rtheta', 'polar-lr', and 'polar-ltheta'
# Note that 'polar-rr' and 'polar-rtheta' refer to the radius and angle

asl.df['polar-rr'] = np.sqrt( (asl.df['right-x'] - asl.df['nose-x'])**2 + (asl.df['right-y'] - asl.df['nose-y'])**2)
asl.df['polar-rtheta'] = np.arctan( (asl.df['right-x'] - asl.df['nose-x'])/(asl.df['right-y'] - asl.df['nose-y']) )

asl.df['polar-lr'] = np.sqrt( (asl.df['left-x'] - asl.df['nose-x'])**2 + (asl.df['left-y'] - asl.df['nose-y'])**2)
asl.df['polar-ltheta'] = np.arctan( (asl.df['left-x'] - asl.df['nose-x'])/(asl.df['left-y'] - asl.df['nose-y']) )

In [ ]:
# TODO add features for left, right, x, y differences by one time step, i.e. the "delta" values discussed in the lecture
# Name these 'delta-rx', 'delta-ry', 'delta-lx', and 'delta-ly'

asl.df['delta-rx'] = asl.df['right-x'].diff().fillna(0)
asl.df['delta-ry'] = asl.df['right-y'].diff().fillna(0)
asl.df['delta-lx'] = asl.df['left-x'].diff().fillna(0)
asl.df['delta-ly'] = asl.df['left-y'].diff().fillna(0)

In [ ]:
# TODO add features of your own design, which may be a combination of the above or something else
# Name these whatever you would like

# TODO define a list named 'features_custom' for building the training set

# First, calculate ground normalized features
# Second, calculate deltas on those features
# Use combination of grnd normalized + deltas. 

df_means = asl.df.groupby('speaker').mean()
df_std = asl.df.groupby('speaker').std()

asl.df['grnd-lx-mean']= asl.df['speaker'].map(df_means['grnd-lx'])
asl.df['grnd-rx-mean']= asl.df['speaker'].map(df_means['grnd-rx'])
asl.df['grnd-ly-mean']= asl.df['speaker'].map(df_means['grnd-ly'])
asl.df['grnd-ry-mean']= asl.df['speaker'].map(df_means['grnd-ry'])

asl.df['grnd-lx-std']= asl.df['speaker'].map(df_std['grnd-lx'])
asl.df['grnd-rx-std']= asl.df['speaker'].map(df_std['grnd-rx'])
asl.df['grnd-ly-std']= asl.df['speaker'].map(df_std['grnd-ly'])
asl.df['grnd-ry-std']= asl.df['speaker'].map(df_std['grnd-ry'])

asl.df['norm-grnd-lx'] = (asl.df['grnd-lx'] - asl.df['grnd-lx-mean']) / asl.df['grnd-lx-std']
asl.df['norm-grnd-rx'] = (asl.df['grnd-rx'] - asl.df['grnd-rx-mean']) / asl.df['grnd-rx-std']
asl.df['norm-grnd-ly'] = (asl.df['grnd-ly'] - asl.df['grnd-ly-mean']) / asl.df['grnd-ly-std']
asl.df['norm-grnd-ry'] = (asl.df['grnd-ry'] - asl.df['grnd-ry-mean']) / asl.df['grnd-ry-std']

asl.df['delta-norm-grnd-lx'] = asl.df['norm-grnd-lx'].diff().fillna(0)
asl.df['delta-norm-grnd-rx'] = asl.df['norm-grnd-rx'].diff().fillna(0)
asl.df['delta-norm-grnd-ly'] = asl.df['norm-grnd-ly'].diff().fillna(0)
asl.df['delta-norm-grnd-ry'] = asl.df['norm-grnd-ry'].diff().fillna(0)


In [ ]:
asl.df['polar-rr-mean'] = asl.df['speaker'].map(df_means['polar-rr']) 
asl.df['polar-rr-std'] = asl.df['speaker'].map(df_std['polar-rr'])
asl.df['polar-lr-mean'] = asl.df['speaker'].map(df_means['polar-lr']) 
asl.df['polar-lr-std'] = asl.df['speaker'].map(df_std['polar-lr']) 

asl.df['norm-polar-rr'] = (asl.df['polar-rr'] - asl.df['polar-rr-mean']) / asl.df['polar-rr-std']
asl.df['norm-polar-lr'] = (asl.df['polar-lr'] - asl.df['polar-lr-mean']) / asl.df['polar-lr-std']

In [ ]:
# You can use them separately or combined (more accuracy)
# 
features_polar = ['polar-rr', 'polar-rtheta', 'polar-lr', 'polar-ltheta']
features_norm = ['norm-rx', 'norm-ry', 'norm-lx','norm-ly']
features_delta = ['delta-rx', 'delta-ry', 'delta-lx', 'delta-ly']

features_normalized_ground = ['norm-grnd-lx', 'norm-grnd-rx', 
                              'norm-grnd-ly', 'norm-grnd-ry']

features_normalized_ground_delta = ['delta-norm-grnd-lx', 'delta-norm-grnd-rx', 
                                    'delta-norm-grnd-ly', 'delta-norm-grnd-ry']

features_normalized_polar = ['norm-polar-lr', 'polar-ltheta', 'norm-polar-rr', 'polar-rtheta']


features_rescaled_ground = ['rescaled-grnd-lx', 'rescaled-grnd-rx', 'rescaled-grnd-ly', 'rescaled-grnd-ry']


features_rescaled_ground_delta = ['rescaled-grnd-lx-delta', 'rescaled-grnd-rx-delta', 
                                  'rescaled-grnd-ly-delta', 'rescaled-grnd-ry-delta']


features_rescaled_polar = ['rescaled-polar-lr', 'polar-ltheta', 'rescaled-polar-rr', 'polar-rtheta']
features_rescaled_distance_squared = ['rescaled-distance-squared']

In [ ]:
# Use normalized ground deltas + normalized polar as custom features throughout
# the later experiments.
features_custom = features_normalized_ground_delta + features_normalized_polar

# Full set of normalized features: even better WERs, but worce training time. 
features_full = features_normalized_ground + features_normalized_ground_delta + features_normalized_polar

In [ ]:
df_min = asl.df.groupby('speaker').min()
df_max = asl.df.groupby('speaker').max()

asl.df['grnd-lx-min']= asl.df['speaker'].map(df_min['grnd-lx'])
asl.df['grnd-rx-min']= asl.df['speaker'].map(df_min['grnd-rx'])
asl.df['grnd-ly-min']= asl.df['speaker'].map(df_min['grnd-ly'])
asl.df['grnd-ry-min']= asl.df['speaker'].map(df_min['grnd-ry'])

asl.df['grnd-lx-max']= asl.df['speaker'].map(df_max['grnd-lx'])
asl.df['grnd-rx-max']= asl.df['speaker'].map(df_max['grnd-rx'])
asl.df['grnd-ly-max']= asl.df['speaker'].map(df_max['grnd-ly'])
asl.df['grnd-ry-max']= asl.df['speaker'].map(df_max['grnd-ry'])

asl.df['polar-lr-min'] = asl.df['speaker'].map(df_min['polar-lr'])
asl.df['polar-lr-max'] = asl.df['speaker'].map(df_max['polar-lr'])
asl.df['polar-rr-min'] = asl.df['speaker'].map(df_min['polar-rr'])
asl.df['polar-rr-max'] = asl.df['speaker'].map(df_max['polar-rr'])


asl.df['rescaled-grnd-lx'] = (asl.df['grnd-lx'] - asl.df['grnd-lx-min']) / (asl.df['grnd-lx-max'] - asl.df['grnd-lx-min'])
asl.df['rescaled-grnd-rx'] = (asl.df['grnd-rx'] - asl.df['grnd-rx-min']) / (asl.df['grnd-rx-max'] - asl.df['grnd-rx-min'])
asl.df['rescaled-grnd-ly'] = (asl.df['grnd-ly'] - asl.df['grnd-ly-min']) / (asl.df['grnd-ly-max'] - asl.df['grnd-ly-min'])
asl.df['rescaled-grnd-ry'] = (asl.df['grnd-ry'] - asl.df['grnd-ry-min']) / (asl.df['grnd-ry-max'] - asl.df['grnd-ry-min'])


asl.df['rescaled-grnd-lx-delta'] = asl.df['rescaled-grnd-lx'].diff().fillna(0)
asl.df['rescaled-grnd-rx-delta'] = asl.df['rescaled-grnd-rx'].diff().fillna(0)
asl.df['rescaled-grnd-ly-delta'] = asl.df['rescaled-grnd-ly'].diff().fillna(0)
asl.df['rescaled-grnd-ry-delta'] = asl.df['rescaled-grnd-ry'].diff().fillna(0)

asl.df['rescaled-polar-lr'] = (asl.df['polar-lr'] - asl.df['polar-lr-min']) / (asl.df['polar-lr-max'] - asl.df['polar-lr-min'])
asl.df['rescaled-polar-rr'] = (asl.df['polar-rr'] - asl.df['polar-rr-min']) / (asl.df['polar-rr-max'] - asl.df['polar-rr-min'])




In [ ]:
asl.df['distance-squared'] = (asl.df['grnd-lx'] - asl.df['grnd-rx'])**2 + (asl.df['grnd-ly'] - asl.df['grnd-ry'])**2

d_min = min(asl.df['distance-squared'])
d_max = max(asl.df['distance-squared'])

asl.df['rescaled-distance-squared'] = (asl.df['distance-squared'] - d_min) / (d_max - d_min)



**Question 1:**  What custom features did you choose for the features_custom set and why?

**Answer 1:** Main idea is to use all the improvements already available and try to combine them: 

I'm using normalized ground coordinates + their deltas (8 features total) as a set of custom features.

1. Normalizing ground features
2. Calculating their deltas
3. Storing everything back into `features_custom`

This usually results in better fitted models (but I'll check if they don't overfit too much), but longer training.

<a id='part1_test'></a>
### Features Unit Testing
Run the following unit tests as a sanity check on the defined "ground", "norm", "polar", and 'delta"
feature sets.  The test simply looks for some valid values but is not exhaustive.  However, the project should not be submitted if these tests don't pass.

In [ ]:
import unittest
# import numpy as np

class TestFeatures(unittest.TestCase):

    def test_features_ground(self):
        sample = (asl.df.ix[98, 1][features_ground]).tolist()
        self.assertEqual(sample, [9, 113, -12, 119])

    def test_features_norm(self):
        sample = (asl.df.ix[98, 1][features_norm]).tolist()
        np.testing.assert_almost_equal(sample, [ 1.153,  1.663, -0.891,  0.742], 3)

    def test_features_polar(self):
        sample = (asl.df.ix[98,1][features_polar]).tolist()
        np.testing.assert_almost_equal(sample, [113.3578, 0.0794, 119.603, -0.1005], 3)

    def test_features_delta(self):
        sample = (asl.df.ix[98, 0][features_delta]).tolist()
        self.assertEqual(sample, [0, 0, 0, 0])
        sample = (asl.df.ix[98, 18][features_delta]).tolist()
        self.assertTrue(sample in [[-16, -5, -2, 4], [-14, -9, 0, 0]], "Sample value found was {}".format(sample))
                         
suite = unittest.TestLoader().loadTestsFromModule(TestFeatures())
unittest.TextTestRunner().run(suite)

<a id='part2_tutorial'></a>
## PART 2: Model Selection
### Model Selection Tutorial
The objective of Model Selection is to tune the number of states for each word HMM prior to testing on unseen data.  In this section you will explore three methods: 
- Log likelihood using cross-validation folds (CV)
- Bayesian Information Criterion (BIC)
- Discriminative Information Criterion (DIC) 

##### Train a single word
Now that we have built a training set with sequence data, we can "train" models for each word.  As a simple starting example, we train a single word using Gaussian hidden Markov models (HMM).   By using the `fit` method during training, the [Baum-Welch Expectation-Maximization](https://en.wikipedia.org/wiki/Baum%E2%80%93Welch_algorithm) (EM) algorithm is invoked iteratively to find the best estimate for the model *for the number of hidden states specified* from a group of sample seequences. For this example, we *assume* the correct number of hidden states is 3, but that is just a guess.  How do we know what the "best" number of states for training is?  We will need to find some model selection technique to choose the best parameter.

In [ ]:
import warnings
from hmmlearn.hmm import GaussianHMM

def train_a_word(word, num_hidden_states, features):
    warnings.filterwarnings("ignore", category=DeprecationWarning)
    training = asl.build_training(features)  
    X, lengths = training.get_word_Xlengths(word)
    model = GaussianHMM(n_components=num_hidden_states, n_iter=1000).fit(X, lengths)
    logL = model.score(X, lengths)
    return model, logL

demoword = 'CHOCOLATE'
model, logL = train_a_word(demoword, 3, features_normalized_ground + features_normalized_ground_delta + features_normalized_polar)
print("Number of states trained in model for {} is {}".format(demoword, model.n_components))
print("logL = {}".format(logL))

The HMM model has been trained and information can be pulled from the model, including means and variances for each feature and hidden state.  The [log likelihood](http://math.stackexchange.com/questions/892832/why-we-consider-log-likelihood-instead-of-likelihood-in-gaussian-distribution) for any individual sample or group of samples can also be calculated with the `score` method.

In [ ]:
def show_model_stats(word, model):
    print("Number of states trained in model for {} is {}".format(word, model.n_components))    
    variance=np.array([np.diag(model.covars_[i]) for i in range(model.n_components)])    
    for i in range(model.n_components):  # for each hidden state
        print("hidden state #{}".format(i))
        print("mean = ", model.means_[i])
        print("variance = ", variance[i])
        print()
    
show_model_stats(demoword, model)

##### Try it!
Experiment by changing the feature set, word, and/or num_hidden_states values in the next cell to see changes in values.  

In [ ]:
# warnings.filterwarnings("ignore", category=RuntimeWarning)
my_testword = 'BOOK'
model, logL = train_a_word(my_testword, 3, features_normalized_ground_delta) # Experiment here with different parameters
show_model_stats(my_testword, model)
print("logL = {}".format(logL))

##### Visualize the hidden states
We can plot the means and variances for each state and feature.  Try varying the number of states trained for the HMM model and examine the variances.  Are there some models that are "better" than others?  How can you tell?  We would like to hear what you think in the classroom online.

In [ ]:
%matplotlib inline

In [ ]:
my_testword = 'JOHN'
model, logL = train_a_word(my_testword, 3, features_ground)

print(logL)

In [ ]:
import math
from matplotlib import (cm, pyplot as plt, mlab)

def visualize(word, model):
    """ visualize the input model for a particular word """
    variance=np.array([np.diag(model.covars_[i]) for i in range(model.n_components)])
    figures = []
    for parm_idx in range(len(model.means_[0])):
        xmin = int(min(model.means_[:,parm_idx]) - max(variance[:,parm_idx]))
        xmax = int(max(model.means_[:,parm_idx]) + max(variance[:,parm_idx]))
        fig, axs = plt.subplots(model.n_components, sharex=True, sharey=False, figsize=(10, 10))
        colours = cm.rainbow(np.linspace(0, 1, model.n_components))
        for i, (ax, colour) in enumerate(zip(axs, colours)):
            x = np.linspace(xmin, xmax, 100)
            mu = model.means_[i,parm_idx]
            sigma = math.sqrt(np.diag(model.covars_[i])[parm_idx])
            normpdf = mlab.normpdf(x, mu, sigma)
            ax.plot(x, normpdf, c=colour)
            ax.set_title("{} feature {} hidden state #{}".format(word, parm_idx, i))

            ax.grid(True)
        figures.append(plt)
    for p in figures:
        p.show()
        
visualize(my_testword, model)

#####  ModelSelector class
Review the `ModelSelector` class from the codebase found in the `my_model_selectors.py` module.  It is designed to be a strategy pattern for choosing different model selectors.  For the project submission in this section, subclass `SelectorModel` to implement the following model selectors.  In other words, you will write your own classes/functions in the `my_model_selectors.py` module and run them from this notebook:

- `SelectorCV `:  Log likelihood with CV
- `SelectorBIC`: BIC 
- `SelectorDIC`: DIC

You will train each word in the training set with a range of values for the number of hidden states, and then score these alternatives with the model selector, choosing the "best" according to each strategy. The simple case of training with a constant value for `n_components` can be called using the provided `SelectorConstant` subclass as follow:

In [ ]:
from my_model_selectors import SelectorConstant

training = asl.build_training(features_ground)  # Experiment here with different feature sets defined in part 1
word = 'VEGETABLE' # Experiment here with different words
model = SelectorConstant(training.get_all_sequences(), training.get_all_Xlengths(), word, n_constant=3).select()
print("Number of states trained in model for {} is {}".format(word, model.n_components))

##### Cross-validation folds
If we simply score the model with the Log Likelihood calculated from the feature sequences it has been trained on, we should expect that more complex models will have higher likelihoods. However, that doesn't tell us which would have a better likelihood score on unseen data.  The model will likely be overfit as complexity is added.  To estimate which topology model is better using only the training data, we can compare scores using cross-validation.  One technique for cross-validation is to break the training set into "folds" and rotate which fold is left out of training.  The "left out" fold scored.  This gives us a proxy method of finding the best model to use on "unseen data". In the following example, a set of word sequences is broken into three folds using the [scikit-learn Kfold](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html) class object. When you implement `SelectorCV`, you will use this technique.

In [ ]:
from sklearn.model_selection import KFold

training = asl.build_training(features_ground) # Experiment here with different feature sets
word = 'VEGETABLE' # Experiment here with different words
word_sequences = training.get_word_sequences(word)
split_method = KFold()
for cv_train_idx, cv_test_idx in split_method.split(word_sequences):
    print("Train fold indices:{} Test fold indices:{}".format(cv_train_idx, cv_test_idx))  # view indices of the folds

**Tip:** In order to run `hmmlearn` training using the X,lengths tuples on the new folds, subsets must be combined based on the indices given for the folds.  A helper utility has been provided in the `asl_utils` module named `combine_sequences` for this purpose.

##### Scoring models with other criterion
Scoring model topologies with **BIC** balances fit and complexity within the training set for each word.  In the BIC equation, a penalty term penalizes complexity to avoid overfitting, so that it is not necessary to also use cross-validation in the selection process.  There are a number of references on the internet for this criterion.  These [slides](http://www2.imm.dtu.dk/courses/02433/doc/ch6_slides.pdf) include a formula you may find helpful for your implementation.

The advantages of scoring model topologies with **DIC** over BIC are presented by Alain Biem in this [reference](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.58.6208&rep=rep1&type=pdf) (also found [here](https://pdfs.semanticscholar.org/ed3d/7c4a5f607201f3848d4c02dd9ba17c791fc2.pdf)).  DIC scores the discriminant ability of a training set for one word against competing words.  Instead of a penalty term for complexity, it provides a penalty if model liklihoods for non-matching words are too similar to model likelihoods for the correct word in the word set.

<a id='part2_submission'></a>
### Model Selection Implementation Submission
Implement `SelectorCV`, `SelectorBIC`, and `SelectorDIC` classes in the `my_model_selectors.py` module.  Run the selectors on the following five words. Then answer the questions about your results.

**Tip:** The `hmmlearn` library may not be able to train or score all models.  Implement try/except contructs as necessary to eliminate non-viable models from consideration.

In [ ]:
# autoreload for automatically reloading changes made in my_model_selectors and my_recognizer
%load_ext autoreload
%autoreload 2

from my_model_selectors import SelectorConstant

In [ ]:
words_to_train = ['FISH', 'BOOK', 'VEGETABLE', 'FUTURE', 'JOHN']
import timeit

In [ ]:
# TODO: Implement SelectorCV in my_model_selector.py
warnings.filterwarnings("ignore", category=RuntimeWarning)
from my_model_selectors import SelectorCV

training = asl.build_training(features_custom)  # Experiment here with different feature sets defined in part 1
sequences = training.get_all_sequences()
Xlengths = training.get_all_Xlengths()
for word in words_to_train:
    start = timeit.default_timer()
    model = SelectorCV(sequences, Xlengths, word, 
                    min_n_components=2, max_n_components=15, random_state = 14).select()
    end = timeit.default_timer()-start
    if model is not None:
        print("Training complete for {} with {} states with time {} seconds".format(word, model.n_components, end))
    else:
        print("Training failed for {}".format(word))

In [ ]:
# TODO: Implement SelectorBIC in module my_model_selectors.py
from my_model_selectors import SelectorBIC

training = asl.build_training(features_normalized_ground_delta)  # Experiment here with different feature sets defined in part 1
sequences = training.get_all_sequences()
Xlengths = training.get_all_Xlengths()
for word in words_to_train:
    start = timeit.default_timer()
    model = SelectorBIC(sequences, Xlengths, word, 
                    min_n_components=2, max_n_components=15, random_state = 14).select()
    end = timeit.default_timer()-start
    if model is not None:
        print("Training complete for {} with {} states with time {} seconds".format(word, model.n_components, end))
    else:
        print("Training failed for {}".format(word))

In [ ]:
# TODO: Implement SelectorDIC in module my_model_selectors.py
from my_model_selectors import SelectorDIC

training = asl.build_training(features_normalized_ground_delta)  # Experiment here with different feature sets defined in part 1
sequences = training.get_all_sequences()
Xlengths = training.get_all_Xlengths()
for word in words_to_train:
    start = timeit.default_timer()
    model = SelectorDIC(sequences, Xlengths, word, 
                    min_n_components=2, max_n_components=15, random_state = 14).select()
    end = timeit.default_timer()-start
    if model is not None:
        print("Training complete for {} with {} states with time {} seconds".format(word, model.n_components, end))
    else:
        print("Training failed for {}".format(word))

**Question 2:**  Compare and contrast the possible advantages and disadvantages of the various model selectors implemented.

**Answer 2:** 

All selection methods need some way to cross-validate the model after the initial training to prevent overfitting and select a model that performs sell on train data AND test data. 

- CV selector model splits training data in a train/test sets and thus selects the model that performs better on test set. It's pretty fast to train (though slower than BIC) and it doesn't penalize larger models just for having more states, but for actual performance. 

- BIC selection prefers smaller models that are less likely to overfit and evaluates models with a cheap formula, hence it's very fast and does a good job preventing overfitting in general cases. 

- DIC penalizes models that ouput high probabilities for other signals (i.e can't discriminate other words). DIC takes longer to train but performs well overall.

<a id='part2_test'></a>
### Model Selector Unit Testing
Run the following unit tests as a sanity check on the implemented model selectors.  The test simply looks for valid interfaces  but is not exhaustive. However, the project should not be submitted if these tests don't pass.

In [ ]:
from asl_test_model_selectors import TestSelectors
suite = unittest.TestLoader().loadTestsFromModule(TestSelectors())
unittest.TextTestRunner().run(suite)

<a id='part3_tutorial'></a>
## PART 3: Recognizer
The objective of this section is to "put it all together".  Using the four feature sets created and the three model selectors, you will experiment with the models and present your results.  Instead of training only five specific words as in the previous section, train the entire set with a feature set and model selector strategy.  
### Recognizer Tutorial
##### Train the full training set
The following example trains the entire set with the example `features_ground` and `SelectorConstant` features and model selector.  Use this pattern for you experimentation and final submission cells.



In [ ]:
# autoreload for automatically reloading changes made in my_model_selectors and my_recognizer
%load_ext autoreload
%autoreload 2

from my_model_selectors import SelectorCV, SelectorDIC, SelectorBIC

def train_all_words(features, model_selector):
    training = asl.build_training(features)  # Experiment here with different feature sets defined in part 1
    sequences = training.get_all_sequences()
    Xlengths = training.get_all_Xlengths()
    model_dict = {}
    for word in training.words:
        model = model_selector(sequences, Xlengths, word, 
                        n_constant=3).select()
        model_dict[word]=model
    return model_dict

models = train_all_words(features_ground, SelectorDIC)
print("Number of word models returned = {}".format(len(models)))

##### Load the test set
The `build_test` method in `ASLdb` is similar to the `build_training` method already presented, but there are a few differences:
- the object is type `SinglesData` 
- the internal dictionary keys are the index of the test word rather than the word itself
- the getter methods are `get_all_sequences`, `get_all_Xlengths`, `get_item_sequences` and `get_item_Xlengths`

In [ ]:
test_set = asl.build_test(features_ground)
print("Number of test set items: {}".format(test_set.num_items))
print("Number of test set sentences: {}".format(len(test_set.sentences_index)))

<a id='part3_submission'></a>
### Recognizer Implementation Submission
For the final project submission, students must implement a recognizer following guidance in the `my_recognizer.py` module.  Experiment with the four feature sets and the three model selection methods (that's 12 possible combinations). You can add and remove cells for experimentation or run the recognizers locally in some other way during your experiments, but retain the results for your discussion.  For submission, you will provide code cells of **only three** interesting combinations for your discussion (see questions below). At least one of these should produce a word error rate of less than 60%, i.e. WER < 0.60 . 

**Tip:** The hmmlearn library may not be able to train or score all models.  Implement try/except contructs as necessary to eliminate non-viable models from consideration.

In [2]:
# TODO implement the recognize method in my_recognizer
from my_recognizer import recognize
from asl_utils import show_errors, error_rate

import pickle

In [3]:
# autoreload for automatically reloading changes made in my_model_selectors and my_recognizer
%load_ext autoreload
%autoreload 2

In [ ]:
feature_sets = { "ground": features_ground, 
                "norm": features_norm, 
                "delta": features_delta,
                "polar": features_polar,
                "norm_ground": features_normalized_ground,
                "norm_ground_delta": features_normalized_ground_delta,
                "norm_polar": features_normalized_polar,
                "custom (norm_ground_delta+norm_polar)": features_custom, 
                "all norm features": features_full,
                "rescaled": features_rescaled_ground,
                "rescaled polar": features_rescaled_polar,
                "rescaled ground+polar": features_rescaled_ground + features_rescaled_polar,
                "rescaled ground delta": features_rescaled_ground_delta,
                "rescaled ground delta+polar": features_rescaled_ground_delta + features_rescaled_polar,
                "rescaled ground delta+polar+distance": features_rescaled_ground_delta + 
                                                        features_rescaled_polar + features_rescaled_distance_squared,
                "rescaled full": features_rescaled_ground + features_rescaled_ground_delta + 
                                features_rescaled_polar + features_rescaled_distance_squared }

model_selectors = [SelectorCV, SelectorBIC, SelectorDIC]

# Try loading saved models
training = []
try:
    training = pickle.load(open("data/training.pkl", "rb"))
except:
    pass

for f, features in feature_sets.items():
    for model_selector in model_selectors:
        
        model_key = f + " / " + str(model_selector)
        
        # If we already trained and saved models for this key 
        # then just look them up, don't train them. 
        # We'll have their stats later on. 
        # 
        if any([t["experiment"] == model_key for t in training]):
            print("Model {} already trained, skipping.".format(model_key))
            continue
        
        print("=====================================================================================================")
        print()
        print("Model: {}".format(model_key))
        
        # Train the models 
        # 
        start = timeit.default_timer()
        models = train_all_words(features, model_selector)
        training_time = timeit.default_timer() - start
        
        # Build the test set 
        # 
        test_set = asl.build_test(features)
        
        # Score the models
        # 
        probabilities, guesses = recognize(models, test_set)
        wer = error_rate(guesses, test_set)
        show_errors(guesses, test_set)
        
        training.append({ "experiment": model_key, 
                        "models": models, 
                        "training_time": training_time,
                        "error_rate": wer})

In [ ]:
# Write a dataset of experiments to csv
training_df = pd.DataFrame(training)
# training_df.to_csv("data/training_experiments.csv")

training_df['experiment'] = training_df.experiment.apply(lambda f: f.split('/')[0] + '/ ' + f.split('/')[-1].split('.')[-1][:-2])
training_df['features'] = training_df.experiment.apply(lambda f: f.split('/')[0])
training_df['model_selector'] = training_df.experiment.apply(lambda f: f.split('/')[-1].split('.')[-1])
training_df.to_csv('data/training_experiments_clean.csv')

In [ ]:
import seaborn as sns
sns.set_style('white')

fig, [ax1, ax2] = plt.subplots(figsize=(20, 20), nrows=2, sharex=True)

ax1.set_title("Feature Sets and Model Selectors Perf")

training_df.pivot(index='features', columns='model_selector', values='error_rate').plot.bar(ax=ax1)
ax1.set_ylabel("WER, lower is better")

training_df.pivot(index='features', columns='model_selector', values='training_time').plot.bar(ax=ax2)
ax2.set_ylabel("Training time, lower is better")

plt.show()

In [ ]:
training_df.sort_values('error_rate')[['error_rate', 'features', 'model_selector']].head(1)

In [ ]:
pickle.dump(training, open("data/training.pkl", "wb"))
pickle.dump(asl, open("data/asl.pkl", "wb"))
pickle.dump(features, open("data/features.pkl", "wb"))

**Question 3:**  Summarize the error results from three combinations of features and model selectors.  What was the "best" combination and why?  What additional information might we use to improve our WER?  For more insight on improving WER, take a look at the introduction to Part 4.

**Answer 3:**

The code above runs every possible option of feature sets + model selectors, saves their training times and error rates to a training_df pd dataframe and saves that to `data/training_experiments.csv`.

The models are saved to a pkl file so that you don't have to train them from scratch.

**Best set of features + model selectors I've tried is rescaled ground delta+polar+distance with SelectorDIC at 37.6% Error rate**

Other ways to improve WER further: 

1. Feature engineering. You can probably squeeze a couple more % out of this dataset with additional features. For example, we could use diff between 2 frames, not 1, or think of some other normalization / scaling technique. This could take just a couple hours, but won't affect the performance dramatically. A 2-3% improvement is probably still possible.
2. Does anyone please have more data? No? Alright ;( 
3. Use language models! A bit more on that: 

My first intuition was to take 3gram models with `arpa` and just use my best features/model selectors combo with them in the following way: 
1. For each sentence: 
2. For each word, take 5 most probable word guesses and their probabilities
3. For each phrase of 4 words within the sentence, also calculate joint log probability of this sentence. This will take 4\*\*5 joint probabilities. Then recalculate everything and for each word probability add the joint sentence probability we calculated with some weight Alpha that we'll need to experiment with.
Done. 

But then I understood that this way I'm discarding all the word probabilities that are under the first 5 but are very much likely in the language model. 

To solve this, we can think of each sentence as a graph: sentence start => word guesses for each word interconnected with the next word layer => sentence end. 

We can then solve the graph path search problem with a UCS algorithm. Path cost for each path would be (joint log probs of this sentence version plus word probs on the way) to the power of -1 (we need inverse of this number to optimize for min path cost). 

That's it, solving with UCS for min path cost would give us a most probable sentence.

<a id='part3_test'></a>
### Recognizer Unit Tests
Run the following unit tests as a sanity check on the defined recognizer.  The test simply looks for some valid values but is not exhaustive. However, the project should not be submitted if these tests don't pass.

In [ ]:
from asl_test_recognizer import TestRecognize
suite = unittest.TestLoader().loadTestsFromModule(TestRecognize())
unittest.TextTestRunner().run(suite)

<a id='part4_info'></a>
## PART 4: (OPTIONAL)  Improve the WER with Language Models
We've squeezed just about as much as we can out of the model and still only get about 50% of the words right! Surely we can do better than that.  Probability to the rescue again in the form of [statistical language models (SLM)](https://en.wikipedia.org/wiki/Language_model).  The basic idea is that each word has some probability of occurrence within the set, and some probability that it is adjacent to specific other words. We can use that additional information to make better choices.

##### Additional reading and resources
- [Introduction to N-grams (Stanford Jurafsky slides)](https://web.stanford.edu/class/cs124/lec/languagemodeling.pdf)
- [Speech Recognition Techniques for a Sign Language Recognition System, Philippe Dreuw et al](https://www-i6.informatik.rwth-aachen.de/publications/download/154/Dreuw--2007.pdf) see the improved results of applying LM on *this* data!
- [SLM data for *this* ASL dataset](ftp://wasserstoff.informatik.rwth-aachen.de/pub/rwth-boston-104/lm/)

##### Optional challenge
The recognizer you implemented in Part 3 is equivalent to a "0-gram" SLM.  Improve the WER with the SLM data provided with the data set in the link above using "1-gram", "2-gram", and/or "3-gram" statistics. The `probabilities` data you've already calculated will be useful and can be turned into a pandas DataFrame if desired (see next cell).  
Good luck!  Share your results with the class!

In [4]:
# Load the required libs

import arpa
from my_recognizer import recognize_with_lang
from my_sentence_recognizer import SentenceProblem
from aimacode.search import uniform_cost_search

In [5]:
# Most of the data is saved before this moment to pkl files. 
# If you reeopen the notebook and are just trying to reproduce the results, 
# you can just load data from pkl to save some time
# 

training = pickle.load(open("data/training.pkl", "rb"))
asl = pickle.load(open("data/asl.pkl", "rb"))
features = pickle.load(open("data/features.pkl", "rb"))
language_model = arpa.loadf("lsm_data/ukn.3.lm")[0]

models = training[44]['models']
test_set = asl.build_test(features)

probs = pickle.load(open("data/probs.pkl", "rb"))
guesses = pickle.load(open("data/guesses.pkl", "rb"))

if not probs or not guesses:
    probs, guesses = recognize(models, test_set)

In [ ]:
pickle.dump(probs, open("data/probs.pkl", "wb") )
pickle.dump(guesses, open("data/guesses.pkl", "wb" ))

In [8]:
# Test if sentence solver doesn't screw up 
# the predictions

sentence = test_set.sentences_index[2]
problem = SentenceProblem(sentence, probs, language_model)
s = uniform_cost_search(problem)

s.solution()

[(1, 0, 'JOHN', 203.67532223469993, ' JOHN'),
 (2, 1, 'WRITE', 232.59337886059345, ' JOHN WRITE'),
 (3, 2, 'HOMEWORK', 285.12193633529273, ' JOHN WRITE HOMEWORK')]

In [21]:
# This sentence is a bit harder, the last word should be CAN, not TOY.
# 

sentence = test_set.sentences_index[7]
problem = SentenceProblem(sentence, probs, language_model)
s = uniform_cost_search(problem)

s.solution()

[(1, 3, 'JOHN', 378.61689522807166, ' JOHN'),
 (2, 4, 'CAN', 81.245432490897784, ' JOHN CAN'),
 (3, 5, 'GO', 94.746341416838902, ' JOHN CAN GO'),
 (4, 6, 'CAN', 115.78279700487232, 'JOHN CAN GO CAN')]

In [ ]:
# Recognize wit lang basically runs the previous recognizer, builds
# itself a list of probs and guesses, and then runs a sentence model 
# for each sentence in the test_set. 
# 
p2, g2 = recognize_with_lang(models, test_set, language_model)

Running a UCS for sentence: [0, 1, 2]
Running a UCS for sentence: [3, 4, 5, 6]
Updating guess value from TOY to CAN
Running a UCS for sentence: [7, 8, 9, 10]
Updating guess value from CAN to GO2
Running a UCS for sentence: [11, 12, 13, 14, 15, 16, 17, 18]


In [ ]:
show_errors(g2, test_set)

In [ ]:
language_model.log_s("<s> JOHN WRITE HOMEWORK </s>")

In [ ]:
language_model.log_s("JOHN WRITE HOMEWORK </s>")

In [ ]:
language_model.log_s("JOHN WRITE VIDEOTAPE")